## Import Dependencies

In [1]:
# Import for Database
from config import password
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import psycopg2
import pandas as pd
%matplotlib inline

## Getting data from PostgreSQL

In [2]:
# config.py will need to be created with password = <postgresql server password>
db_string = f"postgresql+psycopg2://postgres:" + password + "@127.0.0.1:5432/final_project"  # "final_project" will need to be replaced witht the server name
engine = create_engine(db_string)  
inspector = inspect(engine)
inspector.get_table_names()

['stats', 'betting']

In [3]:
connection = engine.connect
session = Session(engine)
engine.execute("SELECT * from stats")

In [4]:
col_names_list = []

#i = 0
for i in range(len(inspector.get_columns('stats'))):
    col_names_list.append(inspector.get_columns('stats')[i]['name'])
    
for i in range(len(inspector.get_columns('betting'))):
    col_names_list.append(inspector.get_columns('betting')[i]['name'])
    
print(col_names_list)

['id', 'div', 'date', 'hometeam', 'awayteam', 'fthg', 'ftag', 'ftr', 'hthg', 'htag', 'htr', 'referee', 'hs', 'ass', 'hst', 'ast', 'hf', 'af', 'hc', 'ac', 'hy', 'ay', 'hr', 'ar', 'id', 'div', 'b365h', 'b365d', 'b365a', 'vch', 'vcd', 'vca']


In [5]:
df = pd.DataFrame(columns = col_names_list)

In [6]:
# Inner join for wines and regions
import sys
join_db = engine.execute("SELECT * from stats inner join betting on stats.id = betting.id")
for record in join_db:
    record_series = pd.Series(list(record), index = df.columns)
    
    df = df.append(record_series, ignore_index=True)
    #print(list(record))

In [7]:
df.head()

,id,div,date,hometeam,awayteam,fthg,ftag,ftr,hthg,htag,...,hr,ar,id,div,b365h,b365d,b365a,vch,vcd,vca
0,0,E0,2007-08-11,Aston Villa,Liverpool,1,2,A,0,1,...,0,0,0,E0,4,3.25,1.9,3.5,3.2,2
1,1,E0,2007-08-11,Bolton,Newcastle,1,3,A,0,3,...,0,0,1,E0,2.5,3.2,2.75,2.3,3.25,2.9
2,2,E0,2007-08-11,Derby,Portsmouth,2,2,D,1,1,...,0,0,2,E0,2.8,3.25,2.4,2.7,3.25,2.4
3,3,E0,2007-08-11,Everton,Wigan,2,1,H,1,0,...,0,0,3,E0,1.66,3.4,5.5,1.7,3.4,5
4,4,E0,2007-08-11,Middlesbrough,Blackburn,1,2,A,1,0,...,0,0,4,E0,2.37,3.25,2.87,2.5,3.25,2.65


## Changing Columns names back to original source

In [8]:
df.columns = df.columns.str.upper()
df.head()

,ID,DIV,DATE,HOMETEAM,AWAYTEAM,FTHG,FTAG,FTR,HTHG,HTAG,...,HR,AR,ID,DIV,B365H,B365D,B365A,VCH,VCD,VCA
0,0,E0,2007-08-11,Aston Villa,Liverpool,1,2,A,0,1,...,0,0,0,E0,4,3.25,1.9,3.5,3.2,2
1,1,E0,2007-08-11,Bolton,Newcastle,1,3,A,0,3,...,0,0,1,E0,2.5,3.2,2.75,2.3,3.25,2.9
2,2,E0,2007-08-11,Derby,Portsmouth,2,2,D,1,1,...,0,0,2,E0,2.8,3.25,2.4,2.7,3.25,2.4
3,3,E0,2007-08-11,Everton,Wigan,2,1,H,1,0,...,0,0,3,E0,1.66,3.4,5.5,1.7,3.4,5
4,4,E0,2007-08-11,Middlesbrough,Blackburn,1,2,A,1,0,...,0,0,4,E0,2.37,3.25,2.87,2.5,3.25,2.65


In [9]:
# Changing the few columns back to Proper
df.rename(columns= {'DATE': 'Date', 'HOMETEAM': 'HomeTeam', 'AWAYTEAM': 'AwayTeam', 'REFEREE': 'Referee', 'ASS': 'AS' },inplace=True)
df.columns

Index(['ID', 'DIV', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF',
       'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'ID', 'DIV', 'B365H', 'B365D',
       'B365A', 'VCH', 'VCD', 'VCA'],
      dtype='object')

In [10]:
# Triming blank spaces from the columns
df['FTR'] = df['FTR'].str.replace(" ","")
df['HTR'] = df['HTR'].str.replace(" ","")
df['HomeTeam'] = df['HomeTeam'].str.strip()
df['AwayTeam'] = df['AwayTeam'].str.strip()

### Keys for Dataframe
 - Div = League Division
 - Date = Match Date
 - HomeTeam = Home Team
 - AwayTeam = Away Team
 - FTHG = Full Time Home Team Goals
 - FTAG = Full Time Away Team Goals
 - FTR = Full Time Result (H=Home Win, D=Draw, A=Away Win)
 - HTHG = Half Time Home Team Goals
 - HTAG = Half Time Away Team Goals
 - HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)
 - Referee = Match Referee
 - HS = Home Team Shots
 - AS = Away Team Shots
 - HST = Home Team Shots on Target
 - AST = Away Team Shots on Target
 - HF = Home Team Fouls Committed
 - AF = Away Team Fouls Committed
 - HC = Home Team Corners
 - AC = Away Team Corners
 - HY = Home Team Yellow Cards
 - AY = Away Team Yellow Cards
 - HR = Home Team Red Cards
 - AR = Away Team Red Cards
 - B365H = Bet365 home win odds
 - B365D = Bet365 draw odds
 - B365A = Bet365 away win odds
 - VCH = VC Bet home win odds
 - VCD = VC Bet draw odds
 - VCA = VC Bet away win odds


## Data Exploration

In [11]:
#what is the win rate for the home team?

# Total number of matches.
n_matches = df.shape[0]

# Calculate number of features. -1 because we are saving one as the target variable (win/lose/draw)
n_features = df.shape[1] - 1

# Calculate matches won.
n_homewins = len(df[df.FTR == 'H'])
n_awaywins = len(df[df.FTR == 'A'])
n_draws = len(df[df.FTR == 'D'])

avg_home_scored = df.FTHG.sum()*1.0 / n_matches
avg_away_scored = df.FTAG.sum()*1.0 / n_matches
avg_home_conceded = avg_away_scored
avg_away_conceded = avg_home_scored



# Calculate win rate
home_win_rate = (float(n_homewins) / (n_matches)) * 100
away_win_rate = (float(n_awaywins) / (n_matches)) * 100
draw_rate = (float(n_draws) / (n_matches)) * 100

# Print the results
print ("Total number of matches: {}".format(n_matches))
print ("Number of features: {}".format(n_features))
print ("Number of matches won by home team: {}".format(n_homewins))
print ("Number of matches won by away team: {}".format(n_awaywins))
print ("Win rate of home team: {:.2f}%".format(home_win_rate))
print ("Win rate of away team: {:.2f}%".format(away_win_rate))
print ("Draw rate of match: {:.2f}%".format(draw_rate))

print ("Average number of goals at home: {:.2f}".format(avg_home_scored))
print ("Average number of goals away: {:.2f}".format(avg_away_scored))
print ("Average number of goals conceded at home: {:.2f}".format(avg_home_conceded))
print ("Average number of goals conceded away: {:.2f}".format(avg_away_conceded))


Total number of matches: 5700
Number of features: 31
Number of matches won by home team: 2594
Number of matches won by away team: 1734
Win rate of home team: 45.51%
Win rate of away team: 30.42%
Draw rate of match: 24.07%
Average number of goals at home: 1.54
Average number of goals away: 1.20
Average number of goals conceded at home: 1.20
Average number of goals conceded away: 1.54


## Preparing the Data for Machine Learning

In [12]:
data = df.copy()
# drop columns that are significant
data.drop(columns = ["ID", "DIV", "Date",'Referee', "B365H", "B365D", "B365A", "VCH", "VCD", "VCA"], axis=1, inplace=True)
data

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,Aston Villa,Liverpool,1,2,A,0,1,A,10,17,6,7,18,11,4,2,4,2,0,0
1,Bolton,Newcastle,1,3,A,0,3,A,13,7,9,5,15,16,4,3,1,1,0,0
2,Derby,Portsmouth,2,2,D,1,1,D,12,12,5,6,14,17,6,6,1,2,0,0
3,Everton,Wigan,2,1,H,1,0,H,12,14,8,4,8,13,6,2,0,0,0,0
4,Middlesbrough,Blackburn,1,2,A,1,0,H,10,4,6,4,16,16,13,3,3,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5695,Crystal Palace,Man United,1,0,H,1,0,H,6,10,3,4,12,22,3,6,2,4,0,0
5696,Leicester,Southampton,4,1,H,0,0,D,12,7,6,2,10,5,3,3,0,1,0,0
5697,Liverpool,Wolves,3,1,H,1,1,D,29,7,8,5,6,3,5,3,1,0,0,0
5698,Man City,Aston Villa,3,2,H,0,1,A,24,4,5,2,5,11,13,1,0,1,0,0


In [14]:
data['HomeTeam'].unique()

array(['Aston Villa', 'Bolton', 'Derby', 'Everton', 'Middlesbrough',
       'Sunderland', 'West Ham', 'Arsenal', 'Chelsea', 'Man United',
       'Tottenham', 'Birmingham', 'Fulham', 'Man City', 'Portsmouth',
       'Reading', 'Wigan', 'Newcastle', 'Blackburn', 'Liverpool', 'Hull',
       'Stoke', 'West Brom', 'Wolves', 'Burnley', 'Blackpool', 'QPR',
       'Swansea', 'Norwich', 'Southampton', 'Crystal Palace', 'Cardiff',
       'Leicester', 'Bournemouth', 'Watford', 'Huddersfield', 'Brighton',
       'Sheffield United', 'Leeds', 'Brentford'], dtype=object)

In [15]:
Team_dict ={'Aston Villa': 0, 'Bolton': 1,'Derby': 2,'Everton': 3,'Middlesbrough': 4,'Sunderland': 5,
            'West Ham': 6,'Arsenal': 7, 'Chelsea': 8, 'Man United': 9, 'Tottenham': 10, 'Birmingham': 11,'Fulham': 12,
            'Man City': 13, 'Portsmouth': 14, 'Reading': 15, 'Wigan': 16, 'Newcastle': 17, 'Blackburn': 18, 
            'Liverpool': 19, 'Hull': 20, 'Stoke': 21, 'West Brom': 22, 'Wolves': 23, 'Burnley': 24, 'Blackpool': 25, 
            'QPR': 26, 'Swansea': 27, 'Norwich': 28, 'Southampton': 29, 'Crystal Palace': 30,  'Cardiff': 31,
            'Leicester': 32, 'Bournemouth': 33, 'Watford': 34,  'Huddersfield': 35, 'Brighton': 36,'Sheffield United': 37, 
            'Leeds': 38, 'Brentford': 39}
data = data.replace({'HomeTeam': Team_dict})
data = data.replace({'AwayTeam': Team_dict})
data.head()


,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,0,19,1,2,0,0,1,0,10,17,6,7,18,11,4,2,4,2,0,0
1,1,17,1,3,0,0,3,0,13,7,9,5,15,16,4,3,1,1,0,0
2,2,14,2,2,0,1,1,0,12,12,5,6,14,17,6,6,1,2,0,0
3,3,16,2,1,1,1,0,1,12,14,8,4,8,13,6,2,0,0,0,0
4,4,18,1,2,0,1,0,1,10,4,6,4,16,16,13,3,3,4,0,0


In [16]:
pd.set_option('display.max_columns', None)
data.dtypes

HomeTeam     int64
AwayTeam     int64
FTHG        object
FTAG        object
FTR          int32
HTHG        object
HTAG        object
HTR          int32
HS          object
AS          object
HST         object
AST         object
HF          object
AF          object
HC          object
AC          object
HY          object
AY          object
HR          object
AR          object
dtype: object

In [17]:
data = data[['HomeTeam', 'AwayTeam', 'FTR', 'HTHG', 'HTAG', 'HTR',
       'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR',
       'AR']]
data

,HomeTeam,AwayTeam,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,0,19,0,0,1,0,10,17,6,7,18,11,4,2,4,2,0,0
1,1,17,0,0,3,0,13,7,9,5,15,16,4,3,1,1,0,0
2,2,14,0,1,1,0,12,12,5,6,14,17,6,6,1,2,0,0
3,3,16,1,1,0,1,12,14,8,4,8,13,6,2,0,0,0,0
4,4,18,0,1,0,1,10,4,6,4,16,16,13,3,3,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5695,30,9,1,1,0,1,6,10,3,4,12,22,3,6,2,4,0,0
5696,32,29,1,0,0,0,12,7,6,2,10,5,3,3,0,1,0,0
5697,19,23,1,1,1,0,29,7,8,5,6,3,5,3,1,0,0,0
5698,13,0,1,0,1,0,24,4,5,2,5,11,13,1,0,1,0,0


## Modeling the Data

In [18]:
#Import for ML
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

In [72]:
# Split our preprocessed data into our features and target arrays
y = data["FTR"].values
X = data.drop("FTR",axis= 1)

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y, test_size= )

In [73]:
X_train.shape, y_train.shape
X_test.shape, y_test.shape

((5643, 17), (5643,))

## Decision Tree

In [21]:
# Create a StandardScaler instance
scaler = StandardScaler()


# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)


# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Creating the decision tree classifier instance
DTC = tree.DecisionTreeClassifier(random_state=0)

# Fitting the model
DTB = DTC.fit(X_train_scaled, y_train)

array([1, 1, 0, ..., 0, 1, 0])

In [45]:
# Making predictions using the testing data
dtc_predictions = DTB.predict(X_test_scaled)
dtc_predictions

dtc_results = pd.DataFrame({
    "Prediction": y_pred, 
    "Actual": y_test
}).reset_index(drop=True)
dtc_results

,Prediction,Actual
0,0,0
1,0,0
2,0,1
3,0,1
4,0,1
...,...,...
1420,0,1
1421,1,1
1422,0,0
1423,0,1


In [26]:
# Calculating the confusion matrix
dt_cm = confusion_matrix(y_test, predictions)
dt_cm_df = pd.DataFrame(
    dt_cm, index=["Away", 'Home'], columns=["Predicted Away", 'Predicted Home']
)

# Calculating the accuracy score
dt_acc_score = accuracy_score(y_test, predictions)

In [27]:
# Displaying results
print("Confusion Matrix")
display(dt_cm_df)
print(f"Accuracy Score : {dt_acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted Away,Predicted Home
Away,557,220
Home,185,463


Accuracy Score : 0.7157894736842105
Classification Report
              precision    recall  f1-score   support

           0       0.75      0.72      0.73       777
           1       0.68      0.71      0.70       648

    accuracy                           0.72      1425
   macro avg       0.71      0.72      0.71      1425
weighted avg       0.72      0.72      0.72      1425



## SVM Model

In [28]:
# Instantiate a linear SVM model
SVM = SVC(kernel='sigmoid')

# Fit the data
SVM.fit(X_train, y_train)

SVC(kernel='sigmoid')

In [43]:
# Make predictions using the test data
y_pred = SVM.predict(X_test)
svm_results = pd.DataFrame({
    "Prediction": y_pred, 
    "Actual": y_test
}).reset_index(drop=True)
svm_results


,Prediction,Actual
0,0,0
1,0,0
2,0,1
3,0,1
4,0,1
...,...,...
1420,0,1
1421,1,1
1422,0,0
1423,0,1


In [41]:
# Calculating the confusion matrix.
svmcm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
svmcm_df = pd.DataFrame(
    svmcm, index=["1", "2"], columns=["Predicted 1", "Predicted 2"])


In [42]:
# Calculating the accuracy score.
svm_acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix")
display(svmcm_df)
print(f"Accuracy Score : {svm_acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 1,Predicted 2
1,557,220
2,185,463


Accuracy Score : 0.7157894736842105
Classification Report
              precision    recall  f1-score   support

           0       0.75      0.72      0.73       777
           1       0.68      0.71      0.70       648

    accuracy                           0.72      1425
   macro avg       0.71      0.72      0.71      1425
weighted avg       0.72      0.72      0.72      1425



## Random Forest Classifier

In [30]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=200, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.789


In [31]:
# Making predictions using the testing data.
rf_predictions = rf_model.predict(X_test_scaled)
rf_predictions

array([1, 0, 0, ..., 0, 1, 0])

In [32]:
# Calculating the confusion matrix.
rfcm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
rfcm_df = pd.DataFrame(
    rfcm, index=["1", "2"], columns=["Predicted 1", "Predicted 2"])


In [34]:
# Calculating the accuracy score.
rf_acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix")
display(rfcm_df)
print(f"Accuracy Score : {rf_acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 1,Predicted 2
1,557,220
2,185,463


Accuracy Score : 0.7157894736842105
Classification Report
              precision    recall  f1-score   support

           0       0.75      0.72      0.73       777
           1       0.68      0.71      0.70       648

    accuracy                           0.72      1425
   macro avg       0.71      0.72      0.71      1425
weighted avg       0.72      0.72      0.72      1425



In [35]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_

# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.13182911323887658, 'HTR'),
 (0.09314536806982039, 'HTHG'),
 (0.09066125561241094, 'HST'),
 (0.06851496173821718, 'HomeTeam'),
 (0.06782384044127045, 'AST'),
 (0.06625544595461964, 'HTAG'),
 (0.06528824256082177, 'AwayTeam'),
 (0.06460576576612222, 'AS'),
 (0.06189236359247714, 'HS'),
 (0.05498835011721622, 'AF'),
 (0.054468950814007475, 'HF'),
 (0.05021926445563883, 'HC'),
 (0.04612697699333534, 'AC'),
 (0.03421739930250543, 'AY'),
 (0.032992227674687846, 'HY'),
 (0.00953861487942811, 'AR'),
 (0.007431858788544475, 'HR')]

## XGBClassifier

In [36]:
# Create a XGB Classifier.
xg_model = XGBClassifier()

# Fitting the model
xg_model = xg_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = xg_model.predict(X_test_scaled)
print(f" XGB Classifier predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 XGB Classifier predictive accuracy: 0.771


In [37]:
# Making predictions using the testing data.
xg_predictions = xg_model.predict(X_test_scaled)
xg_predictions

array([0, 0, 0, ..., 0, 1, 0])

In [38]:
# Calculating the confusion matrix.
xgcm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
xgcm_df = pd.DataFrame(
    xgcm, index=["1", "2"], columns=["Predicted 1", "Predicted 2"])


In [39]:
# Calculating the accuracy score.
xg_acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix")
display(xgcm_df)
print(f"Accuracy Score : {xg_acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 1,Predicted 2
1,557,220
2,185,463


Accuracy Score : 0.7157894736842105
Classification Report
              precision    recall  f1-score   support

           0       0.75      0.72      0.73       777
           1       0.68      0.71      0.70       648

    accuracy                           0.72      1425
   macro avg       0.71      0.72      0.71      1425
weighted avg       0.72      0.72      0.72      1425

